In [1]:
import torch
import copy
import torch.nn.functional as F
from models.tiny_vgg import TinyVGG
from models.module import QLinear,QConv2d, fake_quantize_tensor,quantize_tensor,dequantize_tensor
def cmp(a,b):
    return (a-b).abs().max()
q_b=8
model = TinyVGG(image_size=32,num_classes=10)
model.eval()
model.fuse_bn()
model.quantize(w_num_bits=q_b,a_num_bits=q_b)
model.load_state_dict(torch.load(f'./qat_logs/cifar10/tiny_vgg_w{q_b}a{q_b}/checkpoint_max.pth')['net'])
q_model = copy.deepcopy(model)
q_model.freeze()


/tmp/ipykernel_711466/1156956625.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./qat_logs/cifar10/tiny_vgg_w{q_b}a{q_b}/checkpoint_

In [2]:
q_conv = q_model.stage1.conv1.conv_module
conv = model.stage1.conv1.conv_module

In [ ]:
dummy_input = torch.randn(1, 3, 32, 32)

cmp(model.stage1.conv0(dummy_input), q_model.stage1.conv0(dummy_input))
s1c0=F.relu(model.stage1.conv0(dummy_input))

q_s1c0 = q_model.stage1.conv0.qo.quantize_tensor(s1c0)

s1c1 = (model.stage1.conv1(s1c0))
q_s1c1 = (q_model.stage1.conv1.quantize_inference_integer(q_s1c0))
q_s1c1 = q_model.stage1.conv1.qo.dequantize_tensor(q_s1c1)
cmp(s1c1,q_s1c1)

In [6]:
q_res = F.conv2d(q_s1c0, q_model.stage1.conv1.conv_module.weight, q_model.stage1.conv1.conv_module.bias,
                     q_model.stage1.conv1.conv_module.stride, q_model.stage1.conv1.conv_module.padding,
                     q_model.stage1.conv1.conv_module.dilation, q_model.stage1.conv1.conv_module.groups)


q_weight = model.stage1.conv1.qw.fake_quantize(model.stage1.conv1.conv_module.weight)
bias_scale = model.stage1.conv1.qw.alpha * model.stage1.conv0.qo.alpha
q_bias = fake_quantize_tensor(model.stage1.conv1.conv_module.bias.float(), bias_scale, 32)

res = F.conv2d(s1c0, q_weight, q_bias, 
                    model.stage1.conv1.conv_module.stride, model.stage1.conv1.conv_module.padding,
                    model.stage1.conv1.conv_module.dilation, model.stage1.conv1.conv_module.groups)

cmp(dequantize_tensor(q_res,q_model.stage1.conv1.qi.alpha*q_model.stage1.conv1.qw.alpha),res)

tensor(2.8610e-06, grad_fn=<MaxBackward1>)

In [10]:
cmp(q_res,res)

tensor(3.4242, grad_fn=<MaxBackward1>)

In [11]:
dequantize_tensor(q_model.stage1.conv1.conv_module.bias,q_model.stage1.conv1.qi.alpha*q_model.stage1.conv1.qw.alpha)

tensor([ 1.6467e-01, -5.1973e-01,  6.2731e-01,  6.0706e-01, -1.4062e-04,
         6.6373e-01, -7.5372e-01,  8.1981e-01,  1.5398e-01, -1.2037e-01,
         6.5684e-01,  0.0000e+00,  8.2221e-01, -1.9841e-01, -2.1177e-01,
        -1.0800e-01, -1.4062e-04,  0.0000e+00,  8.8028e-02, -6.4123e-02,
        -5.4982e-02,  4.2313e-01,  1.9827e-01,  1.9574e-01, -2.8321e-01,
        -4.2186e-02, -6.4545e-02,  1.5342e-01,  5.4842e-03,  2.6338e-01,
         3.8235e-01, -1.3921e-02], grad_fn=<MulBackward0>)

In [12]:
model.stage1.conv1.conv_module.bias

Parameter containing:
tensor([ 1.6470e-01, -5.1968e-01,  6.2731e-01,  6.0706e-01, -1.0251e-04,
         6.6372e-01, -7.5367e-01,  8.1986e-01,  1.5396e-01, -1.2041e-01,
         6.5689e-01, -5.6327e-05,  8.2221e-01, -1.9844e-01, -2.1176e-01,
        -1.0802e-01, -1.1255e-04, -6.5899e-05,  8.8008e-02, -6.4127e-02,
        -5.4944e-02,  4.2319e-01,  1.9827e-01,  1.9572e-01, -2.8318e-01,
        -4.2248e-02, -6.4487e-02,  1.5344e-01,  5.5433e-03,  2.6343e-01,
         3.8237e-01, -1.3874e-02], requires_grad=True)

In [4]:
q_conv.weight

Parameter containing:
tensor([[[[ -6., -17.,  -9.],
          [  0.,  66., -35.],
          [ -1., -34.,  24.]],

         [[  0.,   0.,   0.],
          [  0.,   0.,   0.],
          [  0.,   0.,   0.]],

         [[  0.,   0.,   0.],
          [  0.,   0.,   0.],
          [  0.,   0.,   0.]],

         ...,

         [[  0.,  -4.,  -4.],
          [ -2.,  -3.,  -5.],
          [ 24., -19.,   4.]],

         [[  0.,   0.,   0.],
          [  0.,   0.,   0.],
          [  0.,   0.,   0.]],

         [[  1.,   3.,  -3.],
          [  3.,  -3.,  -8.],
          [  1.,   4.,   3.]]],


        [[[-11.,   2.,   0.],
          [ 19.,  32., -21.],
          [ 15.,  15., -40.]],

         [[  0.,   0.,   0.],
          [  0.,   0.,   0.],
          [  0.,   0.,   0.]],

         [[  0.,   0.,   0.],
          [  0.,   0.,   0.],
          [  0.,   0.,   0.]],

         ...,

         [[  2.,   5.,  11.],
          [  6.,   3.,   7.],
          [  4.,  -1.,   8.]],

         [[  0.,   0.,   0

In [7]:
cmp(conv.weight,q_model.stage1.conv1.qw.dequantize_tensor(q_conv.weight))

tensor(0.0034, grad_fn=<MaxBackward1>)

In [9]:
cmp(model.stage1.conv1.qw.fake_quantize(conv.weight),q_model.stage1.conv1.qw.dequantize_tensor(q_conv.weight))

tensor(0., grad_fn=<MaxBackward1>)

In [11]:
q_model.stage1.conv1.qi.alpha

Parameter containing:
tensor(0.0209, requires_grad=True)

In [14]:
model.stage1.conv0.qo.alpha

Parameter containing:
tensor(0.0209, requires_grad=True)